In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [3]:
def save_notebook():
    return display(Javascript("IPython.notebook.save_notebook()"),
                   include=['application/javascript'])

def output_HTML(read_file, output_file):
    from nbconvert import HTMLExporter
    import codecs
    import nbformat
    exporter = HTMLExporter()
    # read_file is '.ipynb', output_file is '.html'
    output_notebook = nbformat.read(read_file, as_version=4)
    output, resources = exporter.from_notebook_node(output_notebook)
    codecs.open(output_file, 'w', encoding='utf-8').write(output)

In [4]:
import sys, os
import glob
sys.path.append('../')

from rankutils.cfgloader import cfgloader

import numpy as np
import pandas

In [5]:
def get_output_name(expcfg_fpath):
    
    expcfg = cfgloader(expcfg_fpath)
    
    tp = expcfg.get('DEFAULT', 'type')
    
    if tp == 'stat':
        featfname_sfx = ''
        expdirname_sfx = "{0:s}".format(expcfg.get('DEFAULT', 'expname'))
        
    elif tp == 'learn':
        featfname_sfx = "{0:s}.npz".format(expcfg.get('DEFAULT', 'expname'))
        expdirname_sfx = "{0:s}.{1:s}".format(expcfg.get('DEFAULT', 'expname'), expcfg.get('IRP', 'classifier'))
        
    elif tp == 'single_learn':
        featfname_sfx = "{0:s}.npz".format(expcfg.get('DEFAULT', 'expname'))
        expdirname_sfx = "{0:s}.{1:s}".format(expcfg.get('DEFAULT', 'expname'), expcfg.get('IRP', 'classifier'))
        
    elif tp == 'block_learn':
        featfname_sfx = "{0:s}.npz".format(expcfg.get('DEFAULT', 'expname'))
        expdirname_sfx = "{0:s}.{1:s}".format(expcfg.get('DEFAULT', 'expname'), expcfg.get('IRP', 'classifier'))
        
    elif tp == 'sequence_label':
        featfname_sfx = "{0:s}.npz".format(expcfg.get('DEFAULT', 'expname'))
        expdirname_sfx = "{0:s}.{1:s}".format(expcfg.get('DEFAULT', 'expname'), expcfg.get('IRP', 'classifier'))
        
    return featfname_sfx, expdirname_sfx

In [6]:
def check_color(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    if val == u'\u2713':
        color = 'green' 
    else:
        color = 'red'
    return 'color: %s' % color

In [7]:
def check_bg(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    if val == u'\u2713':
        color = '#ccffcc' 
    else:
        color = '#ffcccc'
    return 'background-color: %s' % color

In [8]:
# datasets = ['oxford', 'unicamp', 'corel', 'places365', 'vggfaces', 'imagenet']
# variations = [[1, 2, 3, 4, 5], [1, 2, 3, 4, 5],
#               [1, 2],
#               [1, 2, 3],
#               [1, 2],
#               [1, 2, 3]]

datasets = ['unicamp', 'places365', 'imagenet', 'vggfaces']
variations = [[1, 2, 4], [1, 2], [1, 3], [1, 2]]

keys = ["{0:s}_{1:03d}".format(d, v) for d, vl in zip(datasets, variations) for v in vl]

pathcfg = cfgloader('/home/alberto/phD/projects/performance_prediction/ret-mr-learning/source/path_2.cfg')

In [9]:
feature_checklist = []
experiment_checklist = []

expcfglist = glob.glob('/home/alberto/phD/projects/performance_prediction/ret-mr-learning/source/expcfg/main_experiments/k30/*.cfg')
expcfglist.sort()

for expcfg_fpath in expcfglist:
    ff_sfx, exp_sfx = get_output_name(expcfg_fpath)
    
    if ff_sfx and ff_sfx not in feature_checklist:
        feature_checklist.append(ff_sfx)
    
    if exp_sfx not in experiment_checklist:
        experiment_checklist.append(exp_sfx)
    

In [10]:
data_feat = dict(Name=feature_checklist)
data_exp = dict(Name=experiment_checklist)

nf = len(feature_checklist)
ne = len(experiment_checklist)
nk = len(keys)

for i in range(nf):
    ff_sfx = feature_checklist[i]
    
    for j in range(nk):
        key = keys[j]
        if key not in data_feat:
            data_feat[key] = []
        
        feature_dir = pathcfg.get(key, 'feature')
        
        f = glob.glob("{0:s}*{1:s}".format(feature_dir, ff_sfx))
        if f:
            data_feat[key].append(u'\u2713')
        else:
            data_feat[key].append(u'\u2717')
            
for i in range(ne):
    exp_sfx = experiment_checklist[i]
    checks = []
    
    for j in range(nk):
        key = keys[j]
        if key not in data_exp:
            data_exp[key] = []
        
        exp_dir = pathcfg.get(key, 'output')
        
        expfs = glob.glob("{0:s}{1:s}/*.npy".format(exp_dir, exp_sfx))
        #print("{0:s}{1:s}/*.npy".format(exp_dir, exp_sfx))
        if len(expfs) >= 2:
            data_exp[key].append(u'\u2713')
        else:
            data_exp[key].append(u'\u2717')

In [11]:
styles = [
    dict(selector="th", props=[("font-size", "60%"),
                               ("text-align", "left")]),
    dict(selector="caption", props=[("caption-side", "top"), 
                                    ("text-align", "center"),
                                   ("font-size", "130%")])
]

tac = lambda x: "text-align: center"
tal = lambda x: "text-align: left"
fsz = lambda x: "font-size: 140%"

df = pandas.DataFrame(data_feat)
df.style.set_table_styles(styles).applymap(check_color, subset=keys).applymap(check_bg, subset=keys).\
applymap(tac, subset=keys).\
applymap(fsz, subset=keys).\
applymap(tal, subset=['Name']).\
set_caption('Feature Extraction').hide_index()

Name,unicamp_001,unicamp_002,unicamp_004,places365_001,places365_002,imagenet_001,imagenet_003,vggfaces_001,vggfaces_002
pos-learn.0001.irp.deltac_i-64.k30.npz,✓,✓,✓,✓,✓,✓,✓,✓,✓
pos-learn.0002.irp.sdct40.k30.npz,✓,✓,✓,✓,✓,✓,✓,✓,✓
pos-learn.0003.irp.clust-diff-64.k30.npz,✓,✓,✓,✓,✓,✓,✓,✓,✓
pos-learn.0005.irp.accum-rank-jaccard-25-50.k30.npz,✓,✓,✓,✓,✓,✓,✓,✓,✓
pos-learn.0006.irp.cid-jaccard-20-100.k30.npz,✓,✓,✓,✓,✓,✓,✓,✓,✓
pos-learn.0009.irp.fusion_score_1-2-3.k30.npz,✓,✓,✓,✓,✓,✓,✓,✓,✓
pos-learn.0010.irp.fusion_context_5-6.k30.npz,✓,✓,✓,✓,✓,✓,✓,✓,✓
pos-learn.0011.irp.fusion_all.k30.npz,✓,✓,✓,✓,✓,✓,✓,✓,✓
single-learn.0009.irp.fusion_score_1-2-3.k30.npz,✓,✓,✓,✓,✓,✓,✓,✓,✓
single-learn.0010.irp.fusion_context_5-6.k30.npz,✓,✓,✓,✓,✓,✓,✓,✓,✓


In [12]:
df = pandas.DataFrame(data_exp)
df.style.set_table_styles(styles).applymap(check_color, subset=keys).applymap(check_bg, subset=keys).\
applymap(tac, subset=keys).\
applymap(fsz, subset=keys).\
applymap(tal, subset=['Name']).\
set_caption('Experiment').hide_index()

Name,unicamp_001,unicamp_002,unicamp_004,places365_001,places365_002,imagenet_001,imagenet_003,vggfaces_001,vggfaces_002
stat.0001.irp.weibull-fixed.k30,✓,✓,✓,✓,✓,✓,✓,✓,✓
pos-learn.0001.irp.deltac_i-64.k30.svc,✓,✓,✓,✓,✓,✓,✓,✓,✓
pos-learn.0002.irp.sdct40.k30.svc,✓,✓,✓,✓,✓,✓,✓,✓,✓
pos-learn.0003.irp.clust-diff-64.k30.svc,✓,✓,✓,✓,✓,✓,✓,✓,✓
pos-learn.0005.irp.accum-rank-jaccard-25-50.k30.svc,✓,✓,✓,✓,✓,✓,✓,✓,✓
pos-learn.0006.irp.cid-jaccard-20-100.k30.svc,✓,✓,✓,✓,✓,✓,✓,✓,✓
pos-learn.0009.irp.fusion_score_1-2-3.k30.svc,✓,✓,✓,✓,✓,✓,✓,✓,✓
pos-learn.0010.irp.fusion_context_5-6.k30.svc,✓,✓,✓,✓,✓,✓,✓,✓,✓
pos-learn.0011.irp.fusion_all.k30.svc,✓,✓,✓,✓,✓,✓,✓,✓,✓
single-learn.0009.irp.fusion_score_1-2-3.k30.svc,✓,✓,✓,✓,✓,✓,✓,✓,✓
